# 16 1 5

Adding in expected secondary position, which will be used to find the observed secondary lobe closest to the expected secondary position. 

PS3 takes in the following data fields: 
- *TID* — Target ID. If the Target ID is a “D”, then PS3 will process this file, otherwise it will be ignored.
This allows a master database to contain, for instance, single stars used for deconvolution, slit mask
or drift calibrations, etc.
- *DSFN* — Double star (FITS cube) file name.
- *RSFN* — Reference star (FITS) cube file name.
- *ThetaC* — Calculated (input) estimated, predicted, or last catalog reported double star position angle.
- *RhoC* — Calculated (input) estimated, predicted, or last catalog reported double star separation.
- *Delta* — Camera Angle (degrees). Camera orientation angle with respect to the sky.
- *E* — Plate scale (arc seconds / pixel).

For all the FITS data I am working with from April 2014 Kitt Peak run:
- *Delta* = 0 [degrees]
- *E* = 0.01166 [arc seconds / pixel]
